<a href="https://colab.research.google.com/github/ScientistLim/ProjectAW/blob/feature%2Fmodel%2Flstm-transformer/LSTM%26Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
!pip install pytorch-forecasting
!pip instsall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 39.1 MB/s eta 0:00:00
ERROR: unknown command "instsall" - maybe you meant "install"


## Transformer with LSTM

In [4]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Dropout, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, Bidirectional, LSTM
from keras.initializers import Ones, Zeros
from keras.models import Model
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Lambda
import numpy as np
import pandas as pd

In [5]:
# 엑셀 파일 불러오기
file_path = '/content/drive/MyDrive/종설프1_5팀/전처리.ipynb/four-cycle-16months.xlsx'
df = pd.read_excel(file_path)

# 데이터 전처리: 필요한 컬럼 설정 (Time, feed_pressure)
df['time'] = pd.to_datetime(df['time'])
df['time_idx'] = ((df['time'] - df['time'].min()).dt.total_seconds() // 300).astype(int)  # 5분 간격의 시간 인덱스 생성
df['group'] = "feed_pressure"  # 그룹화 컬럼 추가

# 데이터 확인
print(df.head())

# 데이터 길이 확인
total_length = len(df)
print(f"Total data length: {total_length} rows")

                 time  feed_pressure  time_idx          group
0 2024-04-07 23:33:31       0.000000     97161  feed_pressure
1 2024-04-07 23:34:31       0.003304     97161  feed_pressure
2 2024-04-07 23:35:31       0.006620     97162  feed_pressure
3 2024-04-07 23:36:31       0.009646     97162  feed_pressure
4 2024-04-07 23:37:31       0.014007     97162  feed_pressure
Total data length: 971616 rows


In [6]:
# 이상치 탐지 함수
def get_outlier(df=None, column=None, weight=1.5):
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)
    IQR = quantile_75 - quantile_25
    IQR_weight = IQR * weight
    lowest = quantile_25 - IQR_weight * 1.423 / 1.5
    highest = quantile_75 + IQR_weight * 1.6 / 1.5
    outlier_idx = df[column][(df[column] < lowest) | (df[column] > highest)].index
    return outlier_idx

# 이상치 제거
outlier_idx = get_outlier(df=df, column='feed_pressure', weight=1.5)
df.drop(outlier_idx, axis=0, inplace=True)

# 데이터 시퀀스 생성 함수
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

# 시퀀스 길이
sequence_length = 500

# Feed pressure 데이터 시퀀스 생성
X_feed = df["feed_pressure"].values
X_feed_seq, y_feed_seq = create_sequences(X_feed, sequence_length)

# 데이터 분할
X_feed_train, X_feed_test, y_feed_train, y_feed_test = train_test_split(X_feed_seq, y_feed_seq, test_size=0.2, random_state=42)

# 함수 정의

In [42]:
# MultiHeadAttention 클래스 정의
class LayerNormalization(tf.keras.layers.Layer):
    def call(self, inputs):
        mean = tf.keras.backend.mean(inputs, axis=-1, keepdims=True)
        std = tf.keras.backend.std(inputs, axis=-1, keepdims=True)
        return self.gamma * (inputs - mean) / (std + self.eps) + self.beta
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:], initializer=Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:], initializer=Zeros(), trainable=True)
        super(LayerNormalization, self).build(input_shape)

    def call(self, x):
        mean = tf.keras.backend.mean(x, axis=-1, keepdims=True)
        std = tf.keras.backend.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

    def compute_output_shape(self, input_shape):
        return input_shape

In [39]:
class ScaledDotProductAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, attn_dropout=0.1):
        super(ScaledDotProductAttention, self).__init__()
        self.temper = np.sqrt(d_model)
        self.dropout = Dropout(attn_dropout)
        self.softmax = tf.keras.layers.Softmax()

    def call(self, q, k, v, mask=None):
        attn = tf.keras.backend.batch_dot(q, k, axes=[2, 2]) / self.temper
        if mask is not None:
            attn += (-1e+10) * (1 - mask)
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = tf.keras.backend.batch_dot(attn, v)
        return output, attn

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, n_head, d_model, d_k, d_v, dropout):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v
        self.dropout = dropout
        self.qs_layer = Dense(n_head * d_k, use_bias=False)
        self.ks_layer = Dense(n_head * d_k, use_bias=False)
        self.vs_layer = Dense(n_head * d_v, use_bias=False)
        self.attention = ScaledDotProductAttention(d_model, attn_dropout=dropout)
        self.w_o = Dense(d_model)
        self.layer_norm = LayerNormalization()

    def call(self, q, k, v, mask=None):
        batch_size = tf.shape(q)[0]

        # Linear projections
        qs = self.qs_layer(q)  # [batch_size, len_q, n_head * d_k]
        ks = self.ks_layer(k)  # [batch_size, len_k, n_head * d_k]
        vs = self.vs_layer(v)  # [batch_size, len_v, n_head * d_v]

        # Split heads
        qs = tf.reshape(qs, (batch_size, -1, self.n_head, self.d_k))
        ks = tf.reshape(ks, (batch_size, -1, self.n_head, self.d_k))
        vs = tf.reshape(vs, (batch_size, -1, self.n_head, self.d_v))

        # Transpose for attention dot product: [batch_size, n_head, len_q, d_k]
        qs = tf.transpose(qs, [0, 2, 1, 3])
        ks = tf.transpose(ks, [0, 2, 1, 3])
        vs = tf.transpose(vs, [0, 2, 1, 3])

        # Reshape for batch computation
        qs = tf.reshape(qs, (-1, tf.shape(qs)[2], self.d_k))  # [batch_size * n_head, len_q, d_k]
        ks = tf.reshape(ks, (-1, tf.shape(ks)[2], self.d_k))  # [batch_size * n_head, len_k, d_k]
        vs = tf.reshape(vs, (-1, tf.shape(vs)[2], self.d_v))  # [batch_size * n_head, len_v, d_v]

        if mask is not None:
            mask = tf.keras.backend.repeat_elements(mask, self.n_head, axis=0)

        # Scaled dot-product attention
        head, attn = self.attention(qs, ks, vs, mask)

        # Reshape back to original
        head = tf.reshape(head, (batch_size, self.n_head, -1, self.d_v))
        head = tf.transpose(head, [0, 2, 1, 3])
        head = tf.reshape(head, (batch_size, -1, self.n_head * self.d_v))

        # Final linear layer
        outputs = self.w_o(head)
        outputs = Dropout(self.dropout)(outputs)
        return self.layer_norm(outputs), attn


### 모델 정의

In [44]:
class SequentialLSTMTransformer():
    def __init__(self, maxlen, max_features, embed_size, n_head, d_k, d_v):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embed_size = embed_size
        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

    def build_model(self):
        src_seq_input = Input(shape=(self.maxlen,))
        x = Embedding(input_dim=self.max_features, output_dim=self.embed_size)(src_seq_input)

        # LSTM Layers
        lstm_out = Bidirectional(LSTM(128, return_sequences=True))(x)
        lstm_out = Bidirectional(LSTM(64, return_sequences=True))(lstm_out)

        # Transformer Layers
        transformer_out, _ = MultiHeadAttention(n_head=self.n_head, d_model=self.embed_size, d_k=self.d_k, d_v=self.d_v, dropout=0.1)(lstm_out, lstm_out, lstm_out)

        # Global pooling
        avg_pool = GlobalAveragePooling1D()(transformer_out)
        max_pool = GlobalMaxPooling1D()(transformer_out)
        conc = concatenate([avg_pool, max_pool])

        # Fully connected layer
        conc = Dense(64, activation="relu")(conc)
        output = Dense(1, activation="sigmoid")(conc)

        self.model = Model(inputs=src_seq_input, outputs=output)
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return self.model

In [40]:
# LSTM-Transformer 병렬 모델 정의
class ParallelLSTMTransformer():
    def __init__(self, maxlen, max_features, embed_size, n_head, d_k, d_v):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embed_size = embed_size
        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

    def build_model(self):
        src_seq_input = Input(shape=(self.maxlen,))
        x = Embedding(input_dim=self.max_features, output_dim=self.embed_size)(src_seq_input)

        # LSTM Branch
        lstm_branch = Bidirectional(LSTM(128, return_sequences=True))(x)
        lstm_branch = Bidirectional(LSTM(64, return_sequences=True))(lstm_branch)

        # Transformer Branch
        transformer_branch, _ = MultiHeadAttention(n_head=self.n_head, d_model=self.embed_size, d_k=self.d_k, d_v=self.d_v, dropout=0.1)(x, x, x)

        # Concatenate LSTM and Transformer outputs
        merged = concatenate([lstm_branch, transformer_branch])

        # Global pooling
        avg_pool = GlobalAveragePooling1D()(merged)
        max_pool = GlobalMaxPooling1D()(merged)
        conc = concatenate([avg_pool, max_pool])

        # Fully connected layer
        conc = Dense(64, activation="relu")(conc)
        output = Dense(1, activation="sigmoid")(conc)

        self.model = Model(inputs=src_seq_input, outputs=output)
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return self.model

### 모델 생성 및 시각화

In [47]:
# 직렬 LSTM-Transformer 모델 생성 및 TensorBoard 사용 설정
sequential_model = SequentialLSTMTransformer(maxlen=500, max_features=20000, embed_size=300, n_head=3, d_k=64, d_v=64)
model = sequential_model.build_model()

# TensorBoard 설정

example_input = torch.randn(1, 500)  # 입력 크기를 모델에 맞게 조정
# 모델의 구조를 요약하여 출력
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13            │ (None, 500)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_13 (Embedding)  │ (None, 500, 300)       │      6,000,000 │ input_layer_13[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_26          │ (None, 500, 256)       │        439,296 │ embedding_13[0][0]     │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_27          │ (None, 500, 128)       │        164,352 │ bidirectional_26[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_4    │ [(None, None, 300),    │        132,228 │ bidirectional_27[0][0… │
│ (MultiHeadAttention)      │ (None, None, None)]    │                │ bidirectional_27[0][0… │
│                           │                        │                │ bidirectional_27[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 300)            │              0 │ multi_head_attention_… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_1    │ (None, 300)            │              0 │ multi_head_attention_… │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 600)            │              0 │ global_average_poolin… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_58 (Dense)          │ (None, 64)             │         38,464 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_59 (Dense)          │ (None, 1)              │             65 │ dense_58[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 6,774,405 (25.84 MB)

 Trainable params: 6,774,405 (25.84 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# 병렬 LSTM-Transformer 모델 생성 및 TensorBoard 사용 설정
parallel_model = ParallelLSTMTransformer(maxlen=500, max_features=20000, embed_size=300, n_head=3, d_k=64, d_v=64)
model = parallel_model.build_model()

# TensorBoard 설정

example_input = torch.randn(1, 500)  # 입력 크기를 모델에 맞게 조정
# 모델의 구조를 요약하여 출력
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14            │ (None, 500)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_14 (Embedding)  │ (None, 500, 300)       │      6,000,000 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_28          │ (None, 500, 256)       │        439,296 │ embedding_14[0][0]     │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_29          │ (None, 500, 128)       │        164,352 │ bidirectional_28[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_5    │ [(None, None, 300),    │        231,300 │ embedding_14[0][0],    │
│ (MultiHeadAttention)      │ (None, None, None)]    │                │ embedding_14[0][0],    │
│                           │                        │                │ embedding_14[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_3             │ (None, 500, 428)       │              0 │ bidirectional_29[0][0… │
│ (Concatenate)             │                        │                │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 428)            │              0 │ concatenate_3[0][0]    │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_2    │ (None, 428)            │              0 │ concatenate_3[0][0]    │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 856)            │              0 │ global_average_poolin… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_64 (Dense)          │ (None, 64)             │         54,848 │ concatenate_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_65 (Dense)          │ (None, 1)              │             65 │ dense_64[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 6,889,861 (26.28 MB)

 Trainable params: 6,889,861 (26.28 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Lambda  # Lambda 함수 임포트
import matplotlib.pyplot as plt

# Transformer 모델 생성 및 정의 (위 코드 동일)
transformer = Transformer(maxlen=100, max_features=20000, embed_size=300, n_head=3, d_k=64, d_v=64)
model = transformer.build_model()

# 모델 구조 시각화
plot_model(model, to_file='transformer_model.png', show_shapes=True, show_layer_names=True)

# 이미지 표시
img = plt.imread('transformer_model.png')
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.show()


NameError: name 'Transformer' is not defined